<a href="https://colab.research.google.com/github/Lampent/IRFL-website/blob/main/Downloading_OpenImages_Custom_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
cd /content/gdrive/My Drive

/content/gdrive/My Drive


In [4]:
!mkdir OpenImages

In [5]:
cd OpenImages

/content/gdrive/My Drive/OpenImages


In [6]:
!pwd

/content/gdrive/My Drive/OpenImages


In [7]:

# Download required meta-files
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv

!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv

!wget https://storage.googleapis.com/openimages/2018_04/validation/validation-annotations-bbox.csv

!wget https://storage.googleapis.com/openimages/2018_04/test/test-annotations-bbox.csv

--2025-11-10 10:27:11--  https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.207, 172.253.117.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11255 (11K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  10.99K  --.-KB/s    in 0.008s  

2025-11-10 10:27:12 (1.37 MB/s) - ‘class-descriptions-boxable.csv’ saved [11255/11255]

--2025-11-10 10:27:12--  https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.207, 172.253.117.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194033454 (1.1G) [

In [119]:
from PIL import Image
import requests
from io import BytesIO

def get_openimage(image_name: str, mode: str = "train") -> Image.Image:
    """
    Downloads an image from the OpenImages dataset and returns it as a PIL Image.

    Args:
        image_name (str): The image ID or name (without extension), e.g., "000123456789abcdef".
        mode (str): Dataset split - one of "train", "validation", or "test". Default = "train".

    Returns:
        PIL.Image.Image: The loaded image.
    """
    url = f"https://open-images-dataset.s3.amazonaws.com/{mode}/{image_name}"
    resp = requests.get(url, stream=True)
    resp.raise_for_status()
    return Image.open(BytesIO(resp.content)).convert("RGB")


In [95]:
# !pip install awscli

In [26]:
import argparse
import csv
import subprocess
import os
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool as thread_pool
with open('./class-descriptions-boxable.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_list = {rows[1]:rows[0] for rows in reader} # key - object name,  value - object code pairs

In [20]:
import pandas as pd
train = pd.read_csv('./train-annotations-bbox.csv')

In [96]:
selected_df = train[:1000].copy()

In [97]:
code_to_name_dict = {v: k for k, v in dict_list.items()}

In [98]:
selected_df['LabelName'] = train['LabelName'].apply(lambda item: code_to_name_dict[item])

In [99]:
import re
from tqdm import tqdm
import pandas as pd

# human-related keywords
keywords = [
    "person","man","woman","women","child","kid","adult","elder",
    "mom","sister","brother","baby","children","people","pepole","dead"
]
pattern = r'\b(?:' + '|'.join(map(re.escape, keywords)) + r')\b'

# 1️⃣ filter out human-related classes
non_human_df = selected_df[~selected_df["LabelName"].str.contains(pattern, case=False, na=False)]

# 2️⃣ count occurrences of each label per image
counts = (
    non_human_df.groupby(["LabelName", "ImageID"])
    .size()
    .reset_index(name="count")
)

# 3️⃣ create mapping with progress bar
class_map_non_human = {}
labels = counts["LabelName"].unique()

for label in tqdm(labels, desc="Building class map", ncols=90):
    group = counts[counts["LabelName"] == label]
    label_dict = (
        group.groupby("count")["ImageID"]
        .apply(list)
        .to_dict()
    )
    class_map_non_human[label] = label_dict

# import pprint
# sample_preview = {k: list(v.items())[:2] for k, v in list(class_map_non_human.items())[:5]}
# pprint.pprint(sample_preview)


Building class map: 100%|██████████████████████████████| 115/115 [00:00<00:00, 681.63it/s]


In [100]:
# # 4️⃣ if you just want {class: {count: [ids]}} printed nicely:
# import pprint
# pprint.pprint(class_map_non_human['Pillow'])

In [101]:
# !pip install word2number

In [102]:
import re
from word2number import w2n
import pandas as pd
import inflect

p = inflect.engine()

def extract_fields(entry):
    caption = entry.get("caption", "").lower().strip()
    pred_name = entry.get("pred_image_name", "")
    words = re.findall(r"\b[a-z]+\b|\d+", caption)  # simple tokenization (remove punctuation)

    # 1️⃣ edit_type
    edit_type = "add" if "add" in words else "remove" if "remove" in words else None

    # 2️⃣ count: detect either digit or written number
    count = None
    m = re.search(r"\b(\d+)\b", caption)
    if m:
        count = int(m.group(1))
    else:
        try:
            idx = words.index(edit_type) + 1 if edit_type in words else 0
            count = w2n.word_to_num(" ".join(words[idx:idx + 3]))
        except Exception:
            pass

    # 3️⃣ object = all words from the 3rd onward (singularized)
    obj = None
    if len(words) > 2:
        obj_tokens = words[2:]
        obj = " ".join(obj_tokens).strip()
        if obj:
            # singularize last word
            parts = obj.split()
            last = parts[-1]
            singular = p.singular_noun(last)
            if singular:
                parts[-1] = singular
            obj = " ".join(parts)

    # 4️⃣ original_count from filename like "-chip-5.0.png"
    m2 = re.search(r"-[A-Za-z_]+-(\d+(?:\.\d+)?)", pred_name)
    original_count = int(float(m2.group(1))) if m2 else None

    return {
        "edit_type": edit_type,
        "count": count,
        "object": obj,
        "original_count": original_count
    }

def extract_testset_metadata(testset):
    return pd.DataFrame({k: extract_fields(v) for k, v in testset.items()}).T

import json
# Example use:
with open('./test_mapping_instruction.json', 'r') as f:
    testset = json.load(f)

df = extract_testset_metadata(testset)

In [103]:
import pandas as pd

# df columns: image_key, object, edit_type, count, original_count

pairs_map = (
    df.dropna(subset=["object", "original_count", "count", "edit_type"])
      .assign(
          original_count=lambda d: d["original_count"].astype(int),
          requested_count=lambda d: d["count"].astype(int),
      )
      .groupby(["object", "edit_type"])[["original_count", "count"]]
      .apply(lambda g: list(map(tuple, g[["original_count", "count"]].to_numpy())))
      .unstack(fill_value=[])
      .to_dict()
)


In [117]:
import pandas as pd
import inflect
import random
from collections import defaultdict
from typing import Dict, Any, Optional, List

p = inflect.engine()
random.seed(1337)

# ---------------------------- Helpers ----------------------------

def ensure_min_columns(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    if "image_key" not in out.columns:
        if "ref_image_path" in out.columns:
            out["image_key"] = out["ref_image_path"].astype(str)
        elif "pred_image_name" in out.columns:
            out["image_key"] = out["pred_image_name"].astype(str)
        else:
            out["image_key"] = [f"sample_{i:06d}.png" for i in range(len(out))]

    if "ref_image_path" not in out.columns:
        out["ref_image_path"] = out["image_key"]

    # normalize
    if "object" in out.columns:
        out["object"] = out["object"].astype(str).str.strip().str.lower()
    if "edit_type" in out.columns:
        out["edit_type"] = out["edit_type"].astype(str).str.strip().str.lower()

    for c in ("count", "original_count"):
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")

    return out


def pluralize(obj: str, n: int) -> str:
    return p.plural(obj, n) if n != 1 else obj


def choose_source_row(pool_by_obj: Dict[str, List[Dict[str, Any]]],
                      obj: str,
                      orig_count: Optional[int]) -> Optional[Dict[str, Any]]:
    """Pick & remove a reusable row for obj, preferring same original_count."""
    pool = pool_by_obj.get(obj, [])
    # prefer same original_count
    if orig_count is not None:
        for i, r in enumerate(pool):
            oc = r.get("original_count")
            if pd.notna(oc) and int(oc) == int(orig_count):
                return pool.pop(i)
    # otherwise any
    if pool:
        return pool.pop(0)
    return None


# ---------------------------- Main ----------------------------

def build_testset_using_classmap(df: pd.DataFrame, class_map_non_human: Dict[str, Any]) -> Dict[str, Dict[str, Any]]:
    """
    Keeps the same number of rows as df.
    If a row's object is not in allowed set, it is replaced with a random allowed object.
    Reuses existing images when possible; otherwise synthesizes keys.
    """
    df = ensure_min_columns(df)
    df = df.reset_index(drop=True)  # <-- ensure idx is numeric 0..N-1

    # sanity checks
    need = ["object", "edit_type", "count"]
    for col in need:
        if col not in df.columns:
            raise ValueError(f"df is missing required column: {col}")

    allowed = {str(k).strip().lower() for k in class_map_non_human.keys()}
    if not allowed:
        raise ValueError("class_map_non_human is empty or invalid.")

    # Reuse pool: only rows whose object is allowed
    pool_rows = (
        df[df["object"].isin(allowed)]
        .sort_values(["object", "image_key"])
        [["image_key", "object", "ref_image_path", "original_count"]]
        .to_dict("records")
    )
    from collections import defaultdict
    pool_by_obj = defaultdict(list)
    for r in pool_rows:
        pool_by_obj[r["object"]].append(r)

    testset: Dict[str, Dict[str, Any]] = {}

    for idx, row in df.iterrows():  # idx is now guaranteed int
        etype = str(row["edit_type"]).lower()
        req   = int(row["count"]) if pd.notna(row["count"]) else 1
        orig  = None if ("original_count" not in df.columns or pd.isna(row.get("original_count"))) else int(row["original_count"])

        # object: keep if allowed, otherwise replace with random allowed
        obj_src = str(row["object"]).lower()
        obj = obj_src if obj_src in allowed else random.choice(list(allowed))

        # attempt to reuse an image for this (target) object
        picked = choose_source_row(pool_by_obj, obj, orig)

        # caption + names
        verb = "Add" if etype == "add" else "Remove"
        obj_phrase = pluralize(obj, req)
        caption = f"{verb} {req} {obj_phrase}."

        if picked:
            image_key = picked["image_key"]
            ref_image_path = picked.get("ref_image_path", image_key)
            orig_for_name = picked.get("original_count", orig)
        else:
            base_key = f"{obj}_{etype}_{idx:06d}.jpg"
            image_key = base_key
            # ensure uniqueness just in case
            k = 1
            while image_key in testset:
                image_key = f"{obj}_{etype}_{idx:06d}_{k}.jpg"
                k += 1
            ref_image_path = image_key
            orig_for_name = orig

        obj_for_name = obj.replace(" ", "_")
        if orig_for_name is not None and pd.notna(orig_for_name):
            pred_image_name = f"{caption}-{obj_for_name}-{int(orig_for_name)}.jpg"
        else:
            pred_image_name = f"{caption}-{obj_for_name}.jpg"

        testset[image_key] = {
            "caption": caption,
            "ref_image_path": ref_image_path,
            "pred_image_name": pred_image_name,
            "edit_type": etype,
            "requested_count": req,
            "original_count": orig,
            "object": obj
        }

    return testset



# ---------------------------- Usage ----------------------------
# Ensure df has at least: object, edit_type ('add'/'remove'), count
# Optional: original_count, image_key, ref_image_path
#
new_testset = build_testset_using_classmap(df, class_map_non_human)
assert len(new_testset) == len(df)


In [123]:
from PIL import Image
import requests
from io import BytesIO

def get_openimage(image_id: str, mode: str = "train") -> Image.Image:
    """
    Download one valid OpenImages image and return it as a PIL Image.
    Example valid ID: '000002b66c9c498e'
    """
    url = f"https://open-images-dataset.s3.amazonaws.com/{mode}/{image_id}"
    r = requests.get(url, stream=True)
    r.raise_for_status()
    return Image.open(BytesIO(r.content)).convert("RGB")


In [118]:
list(new_testset.keys())[0]

'toy_remove_000000.jpg'

In [124]:
get_openimage(list(new_testset.keys())[0])

HTTPError: 404 Client Error: Not Found for url: https://open-images-dataset.s3.amazonaws.com/train/toy_remove_000000.jpg

In [112]:
new_testset

{'toy_remove_000000.png': {'caption': 'Remove 10 toys.',
  'ref_image_path': 'toy_remove_000000.png',
  'pred_image_name': 'Remove 10 toys.-toy-5.png',
  'edit_type': 'remove',
  'requested_count': 10,
  'original_count': 5,
  'object': 'toy'},
 'cheese_remove_000001.png': {'caption': 'Remove 10 cheeses.',
  'ref_image_path': 'cheese_remove_000001.png',
  'pred_image_name': 'Remove 10 cheeses.-cheese-9.png',
  'edit_type': 'remove',
  'requested_count': 10,
  'original_count': 9,
  'object': 'cheese'},
 'human head_remove_000002.png': {'caption': 'Remove 10 human heads.',
  'ref_image_path': 'human head_remove_000002.png',
  'pred_image_name': 'Remove 10 human heads.-human_head-8.png',
  'edit_type': 'remove',
  'requested_count': 10,
  'original_count': 8,
  'object': 'human head'},
 'cabinetry_remove_000003.png': {'caption': 'Remove 10 cabinetries.',
  'ref_image_path': 'cabinetry_remove_000003.png',
  'pred_image_name': 'Remove 10 cabinetries.-cabinetry-7.png',
  'edit_type': 'remov